# Práctica 2

## Nombre del equipo:
    Master Parsers

## Miembros del equipo:
    - José Gabriel Hernández Castresana
    - Alejandro Guerrero Alvarez
    - Juan Pablo Jiménez Salazar
    - Carlos Flores Vara
    - Armando Guzmán Ramírez

Objetivo: 
    Construir una matriz a partir de los datos obtenidos de la operación

---

A continuación se importan las librerías que vamos a utilizar las cuales son las siguientes:
- NLTK
- re
- numpy
- Pandas
- math
- Counter
- Stopwords

In [9]:
import nltk
import re
import pandas as pd
from collections import Counter
import math
import numpy as np

Crea un diccionario de frequencia de palabras en el texto, tambien lo uso como vocab en el resto del codigo

In [10]:
def makeDicc(og):
    og = og.lower()
    og = re.sub(r"\W"," ",og) # Se usan regular expresions para limpiar el texto de caracteres especiales
    og = re.sub(r"\s+", " ", og)
    toked = nltk.word_tokenize(og)
    from nltk.corpus import stopwords
    p_v = stopwords.words("spanish")
    toked = [w for w in toked if w not in p_v]
    vocab = set(toked)
    vocab = list(vocab)
    return Counter(toked) #counter es una funcion de python para contar la frecuencia de elementos en una lista

Transforma el texto orginal (con ayuda del diccionario) en listas, cada lista es una de las sentences y en la lista estan las palabras de esa linea ya limpiadas

In [11]:
def makeDocsIntoList(dicc,og):
    sentences = nltk.sent_tokenize(og)
    docs = []
    for sentence in sentences:
        rSent = re.sub(r"\W"," ",sentence)
        rSent = re.sub(r"\s+", " ", rSent)
        dn = []
        for word in nltk.word_tokenize(rSent):
            if word in list(dicc.keys()):
                dn.append(word)
        docs.append(dn)
    return docs


Crea la tabla de TF, usando la lista de las sentences y el diccionario

In [12]:
def makeTF(dicc, docsN):
    Tf = pd.DataFrame(index=list(dicc.keys()))
    for i in range (len(docsN)):
        Tf[f"d{i}"] = np.float64(0) # Este np.float64 es necesario ya que la conversion int float no sera automatica en un futuro en pandas
        freqN = Counter(docsN[i])
        freqN = dict(freqN)
        for key, val in freqN.items():
            Tf.loc[key, f"d{i}"] = val/len(docsN[i])
        
    return Tf

Crea tabla IDF

In [13]:
def makeIDF(dicc, docsN):
    IDf = pd.DataFrame(index=list(dicc.keys()))
    IDf['N'] = len(docsN)
    IDf['n(t)'] = 0
    for word in dicc.keys():
        countN = 0
        for doc in docsN:
            if word in doc:
                countN +=1
        IDf.loc[word, 'n(t)'] = countN
    
    IDf['IDF'] = ((IDf['N']/IDf['n(t)']) + 1).apply(math.log)
        
    return IDf

Crea la tabla final idf tf, usando esas dos

In [14]:
def makeIDFTF(tf, idf):
    full = pd.DataFrame(index=list(tf.index.values))
    for i in range (len(tf.columns)):
        full[f"d{i}"] = tf[f"d{i}"] * idf['IDF']

    return full

La operacion se realiza de manera completa

In [18]:
def main(og):
    dicc = makeDicc(og)
    docs = makeDocsIntoList(dicc, og)
    Tf = makeTF(dicc, docs)
    idf = makeIDF(dicc, docs)
    full = makeIDFTF(Tf, idf)

    print ("**** Tf ****")
    print(Tf)
    print()
    print("**** IDF ****")
    print(idf)
    print()
    print("**** Full ****")
    print(full)

    return [Tf, idf, full]

lacosa = """El gato come pescado.
El perro come carne.
El gato y el perro son amigos."""

main(lacosa)

**** Tf ****
               d0        d1        d2
gato     0.333333  0.000000  0.333333
come     0.333333  0.333333  0.000000
pescado  0.333333  0.000000  0.000000
perro    0.000000  0.333333  0.333333
carne    0.000000  0.333333  0.000000
amigos   0.000000  0.000000  0.333333

**** IDF ****
         N  n(t)       IDF
gato     3     2  0.916291
come     3     2  0.916291
pescado  3     1  1.386294
perro    3     2  0.916291
carne    3     1  1.386294
amigos   3     1  1.386294

**** Full ****
               d0        d1        d2
gato     0.305430  0.000000  0.305430
come     0.305430  0.305430  0.000000
pescado  0.462098  0.000000  0.000000
perro    0.000000  0.305430  0.305430
carne    0.000000  0.462098  0.000000
amigos   0.000000  0.000000  0.462098


[               d0        d1        d2
 gato     0.333333  0.000000  0.333333
 come     0.333333  0.333333  0.000000
 pescado  0.333333  0.000000  0.000000
 perro    0.000000  0.333333  0.333333
 carne    0.000000  0.333333  0.000000
 amigos   0.000000  0.000000  0.333333,
          N  n(t)       IDF
 gato     3     2  0.916291
 come     3     2  0.916291
 pescado  3     1  1.386294
 perro    3     2  0.916291
 carne    3     1  1.386294
 amigos   3     1  1.386294,
                d0        d1        d2
 gato     0.305430  0.000000  0.305430
 come     0.305430  0.305430  0.000000
 pescado  0.462098  0.000000  0.000000
 perro    0.000000  0.305430  0.305430
 carne    0.000000  0.462098  0.000000
 amigos   0.000000  0.000000  0.462098]